<a href="https://colab.research.google.com/github/nishantparmar24/Political-Risk/blob/master/BERT_TFHub_Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predicting Sentiment with BERT on TF Hub

In [0]:
from datetime import datetime
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from google.colab import drive
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import csv
import os
import random
import numpy as np

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [0]:
!pip install bert-tensorflow boto3

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from tensorflow import keras
import re

In [0]:
drive.mount('/gdrive')

In [0]:
BASE_DIR = '/gdrive/My Drive/Political Risk Project/'
OUTPUT_DIR = os.path.join(BASE_DIR, 'BertAnalysis', 'ModelOutput5')
# OUTPUT_DIR = 'bert_model'
ACTUAL_PREDS = os.path.join(BASE_DIR, 
                            'Test Data',
                            'PredictionAnalysis', 
                            'reviewed_predictions.csv')

#Data

In [0]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


imdb_train, imdb_test = download_and_load_datasets()

In [0]:
imdb_train = imdb_train[['sentence', 'polarity']]
imdb_test = imdb_test[['sentence', 'polarity']]

In [0]:
embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
# Train on at least 160000 to see the best effects
training_size = 160000
test_portion=.3

In [0]:
corpus = []
# Cleaning the Stanford dataset to remove LATIN1 encoding to make it easier for Python CSV reader
# command: iconv -f LATIN1 -t UTF8 training.1600000.processed.noemoticon.csv -o training_cleaned.csv

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

num_sentences = 0

with open("/tmp/training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        list_item=[]
        label_ = 0 if row[0] == '0' else 1
        list_item.extend([row[5], label_])
        num_sentences = num_sentences + 1
        corpus.append(list_item)

sentences=[]
labels=[]
random.shuffle(corpus)


In [0]:
# train, test = download_and_load_datasets()
data_ = list()
for x in range(training_size):
    row_ = dict()
    row_['sentence'] = corpus[x][0]
    row_['polarity'] = corpus[x][1]
    data_.append(row_)
full_df = pd.DataFrame(data_)


In [0]:
split = int(test_portion * training_size)
tw_test = full_df[:split]
tw_train = full_df[split:training_size]

In [0]:
actual_scores = pd.read_csv(ACTUAL_PREDS)[["headline_text", "actual_score"]]
actual_scores.rename(columns={"headline_text": "sentence", 
                              "actual_score": "polarity"},
                     inplace=True)

In [0]:
actual_scores_neg = actual_scores[actual_scores["polarity"] == 0]
neg_scores = actual_scores_neg.shape[0]
actual_scores_pos = actual_scores[actual_scores["polarity"] == 1]
pos_scores = actual_scores_pos.shape[0]
actual_scores_new = actual_scores_neg[neg_scores-pos_scores:]
actual_scores_new = actual_scores_new.append(actual_scores_pos)
actual_scores_new = actual_scores_new.sample(frac=1)

In [0]:
actual_score_split = int(0.8 * actual_scores_new.shape[0])
train_space = actual_scores_new[:actual_score_split].sample(frac=1)
test_space = actual_scores_new[actual_score_split:].sample(frac=1)

In [0]:
train_space = train_space.append(imdb_train.sample(10000))
test_space = test_space.append(imdb_test.sample(10000))

In [0]:
train_sample = train_space.append(tw_train.sample(10000), sort=False)
test_sample = test_space.append(tw_test.sample(10000), sort=False)

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train_sample.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test_sample.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [0]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

Now let's write code to make predictions on new sentences:

In [0]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
def clean_file_text(file_text):
    ptrn_ = "(\w+\.\s\d+)"
    text_ = re.subn("\n", " ", file_text)[0]
    text_ = re.subn("(\.\s\()", ".(", text_)[0]
    match_num = re.match(ptrn_, text_) or re.search(ptrn_, text_)
    if match_num:
        m_split = re.split(ptrn_, text_)
        for item in m_split:
            m = re.match("(\.\s)(\d+)", item) or re.search("(\.\s)(\d+)", item)
            if m:
                repl_ = "".join(item.split(" "))
                text_ = text_.replace(item, repl_)
    return text_

In [0]:
repo = '/gdrive/My Drive/Political Risk Project/Test Data/PredictionAnalysis'
articles_repo = '/gdrive/My Drive/Political Risk Project/Test Data/'
class_0 = os.path.join(articles_repo, 'Snap Election', '0')
class_1 = os.path.join(articles_repo, 'Snap Election', '1')
articles_0, articles_1 = os.listdir(class_0), os.listdir(class_1)
class_0_files = [f for f in map(lambda x: os.path.join(class_0, x), 
                                  articles_0)]
class_1_files = [f for f in map(lambda x: os.path.join(class_1, x), 
                                  articles_1)]
# all_class_files
file_labels = {"no snap": class_0_files,
               "snap": class_1_files}

In [0]:
def get_sentence_level_prediction(class_):
    occurrences = dict()
    for filename in class_:
        # fpath = os.path.join(class_, filename)
        # print("\nFile path: {}".format(fpath))
        with open(filename, encoding='cp1252') as f:
            file_contents = f.read()
        file_clean = re.subn(r'[“”,]', '', clean_file_text(file_contents))
        file_sentences = re.split('\.\s+', file_clean[0].strip())
        article_prediction = pd.DataFrame(
            getPrediction(pd.Series(file_sentences)))
        article_prediction.rename(
            columns={0: 'sentence', 1: 'logits', 2: 'label'},
            inplace=True)
        occurrences[filename] = article_prediction
    return occurrences

In [0]:
def analyze_file_text(file_dict):
    message = ""
    file_text_collection = pd.DataFrame()
    sentence_collection = pd.DataFrame()
    for label, files in file_dict.items():
        message += "\nPredictions for file classes: {}".format(label)
        occurrences = get_sentence_level_prediction(files)
        collection_ = list()
        sentence_collection_ = list()
        for fn, target in occurrences.items():
            file_id = "{}_{}".format("_".join(label.split(" ")),
                                    fn.split(os.path.sep)[-1])
            file_stats = dict()
            sentence_predictions = dict()
            message+="file: {}".format(file_id)
            file_stats['file'] = file_id
            file_stats['neg_count'] = target[target['label'] == "Negative"].shape[0]
            file_stats['pos_count'] = target[target['label'] == "Positive"].shape[0]
            sentence_predictions['file'] = file_id
            sentence_predictions['negative_predictions'] = list(
                target[target['label'] == "Negative"]["sentence"])
            sentence_predictions['positive_predictions'] = list(
                target[target['label'] == "Positive"]["sentence"])
            message += "\n{} Negative:Positive = {}:{}".format(fn,
                target[target['label'] == "Negative"].shape[0], 
                target[target['label'] == "Positive"].shape[0])
            collection_.append(file_stats)
            sentence_collection_.append(sentence_predictions)

        collection_df = pd.DataFrame(collection_)
        if file_text_collection.empty:
            file_text_collection = collection_df
        else:
            file_text_collection = file_text_collection.append(collection_df)
        
        collection_df = pd.DataFrame(sentence_collection_)
        if sentence_collection.empty:
            sentence_collection = collection_df
        else:
            sentence_collection = sentence_collection.append(collection_df)
    return (file_text_collection, sentence_collection, message)
        # file_text_collection.append(collection_)

In [0]:
# To write the Neg/Pos Sentence predictions
def write_sentence_level_pred(sentence_collection):
    sentence_collection.reset_index(inplace=True)
    sentence_collection.drop(columns=['index'], inplace=True)
    files = list(sentence_collection['file'])
    for file_ in files:
        n = sentence_collection[sentence_collection['file']==file_].squeeze()["negative_predictions"]
        p = sentence_collection[sentence_collection['file']==file_].squeeze()["positive_predictions"]
        n = "\n\n".join(n)
        p = "\n\n".join(p)

        with open(
            os.path.join(
                repo, 'ReutersArticles', 'Negative', file_), 'w') as f:
            f.write(n)
        with open(
            os.path.join(
                repo, 'ReutersArticles', 'Positive', file_), 'w') as f:
            f.write(p)
# write_sentence_level_pred(sentence_collection)

In [0]:
import matplotlib.pyplot as plt


def visualize_sentiment_stats(stats_df, col_list):
    likert_colors = ['white', 'firebrick', 'darkblue']
    dummy = stats_df.copy()
    middles = dummy[col_list[:-1]].sum(axis=1) + dummy[col_list[-1]]
    longest = middles.max()
    complete_longest = dummy.sum(axis=1).max()
    dummy.insert(0, '', (middles - longest).abs())

    dummy.plot.barh(stacked=True, color=likert_colors, edgecolor='none', 
                    legend=False)
    z = plt.axvline(longest, linestyle='--', color='black', alpha=.5)
    z.set_zorder(-1)

    plt.xlim(0, complete_longest)
    xvalues = range(0,complete_longest,10)
    xlabels = [str(x-longest) for x in xvalues]
    plt.xticks(xvalues, xlabels)
    plt.show()


# visualize_sentiment_stats(file_text_stats, file_text_stats.columns)

## Deploy model to S3:
---


In [0]:
def serving_input_fn():
    reciever_tensors = {
        "input_ids": tf.placeholder(dtype=tf.int32,
                                    shape=[1, MAX_SEQ_LENGTH])
    }
    features = {
        "input_ids": reciever_tensors['input_ids'],
        "input_mask": 1 - tf.cast(tf.equal(reciever_tensors['input_ids'], 0), dtype=tf.int32),
        "segment_ids": tf.zeros(dtype=tf.int32, shape=[1, MAX_SEQ_LENGTH]),
        "label_ids": tf.placeholder(tf.int32, [None], name='label_ids')
    }
    return tf.estimator.export.ServingInputReceiver(features, reciever_tensors)
    
estimator._export_to_tpu = False
estimator.export_saved_model(OUTPUT_DIR+"/export", serving_input_fn)

In [0]:
aws_credentials = '/gdrive/My Drive/AWS/AP/credentials'


def get_credentials(param='default'):
    with open(aws_credentials) as f:
        creds = f.read()
    creds_split = creds.split('\n')
    creds_dict = dict()
    prev_key = None
    for line in creds_split:
        if not re.search('=', line):
            match = re.search('\[(.*)\]', line)
            if match:
                key = match.groups()[0]
                creds_dict[key] = dict()
                prev_key = key
        else:
            if prev_key:
                creds_dict[prev_key].update(
                    {line.split('=')[0]:line.split('=')[1]})
    if param in creds_dict.keys():
        return creds_dict[param]


In [0]:
import boto3

creds_ = get_credentials()
S3_UPLOAD_PATH = "s3://eiu-political-risk-nlp-model/bert"
try:
    bucket = re.search("s3://(.+?)/", S3_UPLOAD_PATH).group(1)
    key = re.search("s3://.+?/(.+)", S3_UPLOAD_PATH).group(1)
except:
    print("\033[91m{}\033[00m".format(
        "ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), 
        file=sys.stderr)
s3 = boto3.client("s3", 
                  aws_access_key_id=creds_['aws_access_key_id'], 
                  aws_secret_access_key=creds_['aws_secret_access_key'])
for dirpath, _, filenames in os.walk(OUTPUT_DIR+"/export"):
    for filename in filenames:
        filepath = os.path.join(dirpath, filename)
        filekey = os.path.join(key, filepath[len(OUTPUT_DIR+"/export/"):])
        print("Uploading s3://{}/{} ...".format(bucket, filekey), end = '')
        s3.upload_file(filepath, bucket, filekey)
        print(" ✓")

print("\nUploaded model export directory to " + S3_UPLOAD_PATH)